# Intro to EDSL
This notebook provides example code for base components of [EDSL, an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. Details on the code below are provided in accompanying [slides: How to use EDSL](https://docs.google.com/presentation/d/10GxXhzu_TD09vN0gJhfne0Zum-GF5R-ppzTXb5IUKlU/edit?usp=sharing).

## Technical setup
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. 

## Documentation
Please also see our [documentation page](https://docs.expectedparrot.com/) for tips, tutorials and more demo notebooks on using EDSL.

## Simple example
We start by [selecting a question type](https://docs.expectedparrot.com/en/latest/questions.html) and constructing a question in the relevant template:

In [1]:
from edsl import QuestionMultipleChoice

q = QuestionMultipleChoice(
    question_name = "marvel_movies",
    question_text = "Do you enjoy Marvel movies?",
    question_options = ["Yes", "No", "I do not know"]
)

We administer a question by calling the `run()` method. 
This generates a dataset of `Results` including the model's response to the question:

In [2]:
results = q.run()

results.select("marvel_movies")

answer.marvel_movies
I do not know


## Designing AI agents
We can [create personas for agents](https://docs.expectedparrot.com/en/latest/agents.html) to answer the question:

In [3]:
from edsl import AgentList, Agent

personas = ["comic book collector", "movie critic"]

a = AgentList(
    Agent(traits = {"persona": p}) for p in personas
)

## Selecting language models
We can [select language models](https://docs.expectedparrot.com/en/latest/language_models.html) to generate the responses (in the example above we did not specify a model, so GPT 4 preview was used by default):

In [4]:
from edsl import ModelList, Model

models = ["gpt-4o", "claude-3-5-sonnet-20240620"]

m = ModelList(
    Model(m) for m in ["gpt-4o", "claude-3-5-sonnet-20240620"]
)

## Generating results
We add agents and models to a question when running it:

In [5]:
results = q.by(a).by(m).run()

results.select("model", "persona", "marvel_movies")

model.model,agent.persona,answer.marvel_movies
gpt-4o,comic book collector,Yes
claude-3-5-sonnet-20240620,comic book collector,Yes
gpt-4o,movie critic,Yes
claude-3-5-sonnet-20240620,movie critic,Yes


## Parameterizing questions
We can use `Scenario` objects to [add data or content to questions](https://docs.expectedparrot.com/en/latest/scenarios.html):

In [6]:
q1 = QuestionMultipleChoice(
    question_name = "politically_motivated",
    question_text = """
    Read the following movie review and determine whether it is politically motivated.
    Movie: {{ title }}
    Review: {{ review }}
    """,
    question_options = ["Yes", "No", "I do not know"]
)

EDSL comes with [methods for generating scenarios from many data sources](https://docs.expectedparrot.com/en/latest/scenarios.html), including PDFs, CSVs, docs, images, tables, lists, dicts:

In [7]:
from edsl import Scenario

example_review = {
    "year": 2014,
    "title": "Captain America: The Winter Soldier",
    "review": """
    Part superhero flick, part 70s political thriller. 
    It's a bold mix that pays off, delivering a scathing 
    critique of surveillance states wrapped in spandex 
    and shield-throwing action.    
    """
}

s = Scenario.from_dict(example_review)

In [8]:
results = q1.by(s).by(a).by(m).run()

(
    results.filter("persona == 'movie critic'")
    .sort_by("model")
    .select("model", "year", "title", "politically_motivated")
)

model.model,scenario.year,scenario.title,answer.politically_motivated
claude-3-5-sonnet-20240620,2014,Captain America: The Winter Soldier,No
gpt-4o,2014,Captain America: The Winter Soldier,Yes


## Comments
Questions automatically include a "comment" field.
This can be useful for understanding the context of a response, or debugging a non-response.

In [9]:
(
    results.filter("persona == 'movie critic'")
    .sort_by("model")
    .select("model", "politically_motivated", "politically_motivated_comment")
)

model.model,answer.politically_motivated,comment.politically_motivated_comment
claude-3-5-sonnet-20240620,No,"While the review mentions political themes, it doesn't appear to be politically motivated itself. It's a straightforward assessment of the film's genre blend and thematic content without pushing any particular political agenda."
gpt-4o,Yes,"The review explicitly mentions a ""scathing critique of surveillance states,"" indicating a political perspective on the film's themes."


## Combining questions in a survey
We can [combine questions in a `Survey`](https://docs.expectedparrot.com/en/latest/surveys.html) to administer them together.
Here we create some variations on the above question to compare responses:

In [10]:
from edsl import QuestionYesNo

q2 = QuestionYesNo(
    question_name = "yn",
    question_text = """
    Read the following movie review and determine whether it is politically motivated.
    Movie: {{ title }}
    Review: {{ review }}
    """
)

In [11]:
from edsl import QuestionLinearScale

q3 = QuestionLinearScale(
    question_name = "ls",
    question_text = """
    Read the following movie review and indicate whether it is politically motivated.
    Movie: {{ title }}
    Review: {{ review }}
    """,
    question_options = [0,1,2,3,4,5],
    option_labels = {0:"Not at all", 5:"Very much"}
)

In [12]:
from edsl import QuestionList

q4 = QuestionList(
    question_name = "favorites",
    question_text = "List your favorite Marvel movies.",
    max_list_items = 3
)

## Survey rules & logic
We can [add skip/stop and other rules](https://docs.expectedparrot.com/en/latest/surveys.html), and "memory" of other questions in a survey:

In [13]:
from edsl import Survey

survey = Survey(questions = [q2, q3, q4])

survey = survey.add_stop_rule(q3, "ls < 3")

In [14]:
results = survey.by(s).by(a).by(m).run()

In [15]:
(
    results.filter("persona == 'comic book collector'")
    .select("model", "persona", "yn", "ls", "favorites")
    .print(pretty_labels = {
        "answer.yn": "Yes/No version",
        "answer.ls": "Linear scale version",
        "answer.favorites": "Favorites"
    })
)

model.model,agent.persona,Yes/No version,Linear scale version,Favorites
gpt-4o,comic book collector,Yes,3,"['Guardians of the Galaxy', 'Spider-Man: Into the Spider-Verse', 'Avengers: Endgame']"
claude-3-5-sonnet-20240620,comic book collector,No,2,


## Working with results as datasets
EDSL provides [built-in methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html), e.g., as SQL tables, dataframes:

In [16]:
results.sql("select model, persona, yn, ls, favorites from self")

model,persona,yn,ls,favorites
gpt-4o,comic book collector,Yes,3,"['Guardians of the Galaxy', 'Spider-Man: Into the Spider-Verse', 'Avengers: Endgame']"
claude-3-5-sonnet-20240620,comic book collector,No,2,
gpt-4o,movie critic,Yes,3,"['Avengers: Endgame', 'Black Panther', 'Guardians of the Galaxy']"
claude-3-5-sonnet-20240620,movie critic,No,0,


In [17]:
results.to_pandas()

,answer.yn,answer.ls,answer.favorites,scenario.year,scenario.title,scenario.review,agent.agent_instruction,agent.agent_name,agent.persona,model.frequency_penalty,...,question_options.ls_question_options,question_type.favorites_question_type,question_type.yn_question_type,question_type.ls_question_type,comment.yn_comment,comment.ls_comment,comment.favorites_comment,generated_tokens.ls_generated_tokens,generated_tokens.yn_generated_tokens,generated_tokens.favorites_generated_tokens
0,Yes,3,"['Guardians of the Galaxy', 'Spider-Man: Into ...",2014,Captain America: The Winter Soldier,"\n Part superhero flick, part 70s political...",You are answering questions as if you were a h...,Agent_9,comic book collector,0,...,"[0, 1, 2, 3, 4, 5]",list,yes_no,linear_scale,While the review highlights the film's action ...,The review suggests that the movie delivers a ...,These movies capture the essence of what makes...,3 \nThe review suggests that the movie delive...,Yes\n\nWhile the review highlights the film's ...,"[""Guardians of the Galaxy"", ""Spider-Man: Into ..."
1,No,2,NaN,2014,Captain America: The Winter Soldier,"\n Part superhero flick, part 70s political...",You are answering questions as if you were a h...,Agent_10,comic book collector,0,...,"[0, 1, 2, 3, 4, 5]",list,yes_no,linear_scale,"Comment: As a comic book collector, I don't se...",Task was cancelled.,Task was cancelled.,NaN,"No\n\nComment: As a comic book collector, I do...",NaN
2,Yes,3,"['Avengers: Endgame', 'Black Panther', 'Guardi...",2014,Captain America: The Winter Soldier,"\n Part superhero flick, part 70s political...",You are answering questions as if you were a h...,Agent_11,movie critic,0,...,"[0, 1, 2, 3, 4, 5]",list,yes_no,linear_scale,"The review mentions a ""scathing critique of su...",The review acknowledges the film's critique of...,These films stand out due to their compelling ...,3 \nThe review acknowledges the film's critiq...,"Yes\n\nThe review mentions a ""scathing critiqu...","[""Avengers: Endgame"", ""Black Panther"", ""Guardi..."
3,No,0,NaN,2014,Captain America: The Winter Soldier,"\n Part superhero flick, part 70s political...",You are answering questions as if you were a h...,Agent_12,movie critic,0,...,"[0, 1, 2, 3, 4, 5]",list,yes_no,linear_scale,Comment: This review does not appear to be pol...,Task was cancelled.,Task was cancelled.,NaN,No\n\nComment: This review does not appear to ...,NaN


In [18]:
results.to_csv("marvel_movies_survey.csv")

FileStore(path='marvel_movies_survey.csv', base64_string='YW5zd2VyLnluLGFuc3dlci5scyxhbnN3ZXIuZmF2b3JpdGVzLHNjZW5hcmlvLnllYXIsc2NlbmFyaW8udGl0bGUsc2NlbmFyaW8ucmV2aWV3LGFnZW50LmFnZW50X2luc3RydWN0aW9uLGFnZW50LmFnZW50X25hbWUsYWdlbnQucGVyc29uYSxtb2RlbC5mcmVxdWVuY3lfcGVuYWx0eSxtb2RlbC5sb2dwcm9icyxtb2RlbC5tYXhfdG9rZW5zLG1vZGVsLm1vZGVsLG1vZGVsLnRlbXBlcmF0dXJlLG1vZGVsLnRvcF9wLG1vZGVsLnByZXNlbmNlX3BlbmFsdHksbW9kZWwudG9wX2xvZ3Byb2JzLHByb21wdC55bl91c2VyX3Byb21wdCxwcm9tcHQubHNfdXNlcl9wcm9tcHQscHJvbXB0LmZhdm9yaXRlc19zeXN0ZW1fcHJvbXB0LHByb21wdC5mYXZvcml0ZXNfdXNlcl9wcm9tcHQscHJvbXB0LmxzX3N5c3RlbV9wcm9tcHQscHJvbXB0LnluX3N5c3RlbV9wcm9tcHQscmF3X21vZGVsX3Jlc3BvbnNlLmxzX3Jhd19tb2RlbF9yZXNwb25zZSxyYXdfbW9kZWxfcmVzcG9uc2UubHNfb25lX3VzZF9idXlzLHJhd19tb2RlbF9yZXNwb25zZS5mYXZvcml0ZXNfY29zdCxyYXdfbW9kZWxfcmVzcG9uc2UueW5fb25lX3VzZF9idXlzLHJhd19tb2RlbF9yZXNwb25zZS5sc19jb3N0LHJhd19tb2RlbF9yZXNwb25zZS5mYXZvcml0ZXNfb25lX3VzZF9idXlzLHJhd19tb2RlbF9yZXNwb25zZS5mYXZvcml0ZXNfcmF3X21vZGVsX3Jlc3BvbnNlLHJhd19tb2RlbF9yZXNwb25zZS55bl9yYXdfbW9kZWxfcmVzcG9uc2UscmF3X21vZGVsX3Jlc3BvbnNlLnluX2Nvc3QsaXRlcmF0aW9uLml0ZXJhdGlvbixxdWVzdGlvbl90ZXh0LnluX3F1ZXN0aW9uX3RleHQscXVlc3Rpb25fdGV4dC5sc19xdWVzdGlvbl90ZXh0LHF1ZXN0aW9uX3RleHQuZmF2b3JpdGVzX3F1ZXN0aW9uX3RleHQscXVlc3Rpb25fb3B0aW9ucy55bl9xdWVzdGlvbl9vcHRpb25zLHF1ZXN0aW9uX29wdGlvbnMuZmF2b3JpdGVzX3F1ZXN0aW9uX29wdGlvbnMscXVlc3Rpb25fb3B0aW9ucy5sc19xdWVzdGlvbl9vcHRpb25zLHF1ZXN0aW9uX3R5cGUuZmF2b3JpdGVzX3F1ZXN0aW9uX3R5cGUscXVlc3Rpb25fdHlwZS55bl9xdWVzdGlvbl90eXBlLHF1ZXN0aW9uX3R5cGUubHNfcXVlc3Rpb25fdHlwZSxjb21tZW50LnluX2NvbW1lbnQsY29tbWVudC5sc19jb21tZW50LGNvbW1lbnQuZmF2b3JpdGVzX2NvbW1lbnQsZ2VuZXJhdGVkX3Rva2Vucy5sc19nZW5lcmF0ZWRfdG9rZW5zLGdlbmVyYXRlZF90b2tlbnMueW5fZ2VuZXJhdGVkX3Rva2VucyxnZW5lcmF0ZWRfdG9rZW5zLmZhdm9yaXRlc19nZW5lcmF0ZWRfdG9rZW5zDQpZZXMsMywiWydHdWFyZGlhbnMgb2YgdGhlIEdhbGF4eScsICdTcGlkZXItTWFuOiBJbnRvIHRoZSBTcGlkZXItVmVyc2UnLCAnQXZlbmdlcnM6IEVuZGdhbWUnXSIsMjAxNCxDYXB0YWluIEFtZXJpY2E6IFRoZSBXaW50ZXIgU29sZGllciwiCiAgICBQYXJ0IHN1cGVyaGVybyBmbGljaywgcGFydCA3MHMgcG9saXRpY2FsIHRocmlsbGVyLiAKICAgIEl0J3MgYSBib2xkIG1peCB0aGF0IHBheXMgb2ZmLCBkZWxpdmVyaW5nIGEgc2NhdGhpbmcgCiAgICBjcml0aXF1ZSBvZiBzdXJ2ZWlsbGFuY2Ugc3RhdGVzIHdyYXBwZWQgaW4gc3BhbmRleCAKICAgIGFuZCBzaGllbGQtdGhyb3dpbmcgYWN0aW9uLiAgICAKICAgICIsWW91IGFyZSBhbnN3ZXJpbmcgcXVlc3Rpb25zIGFzIGlmIHlvdSB3ZXJlIGEgaHVtYW4uIERvIG5vdCBicmVhayBjaGFyYWN0ZXIuLEFnZW50XzksY29taWMgYm9vayBjb2xsZWN0b3IsMCxGYWxzZSwxMDAwLGdwdC00bywwLjUsMSwwLDMsIgoKICAgIFJlYWQgdGhlIGZvbGxvd2luZyBtb3ZpZSByZXZpZXcgYW5kIGRldGVybWluZSB3aGV0aGVyIGl0IGlzIHBvbGl0aWNhbGx5IG1vdGl2YXRlZC4KICAgIE1vdmllOiBDYXB0YWluIEFtZXJpY2E6IFRoZSBXaW50ZXIgU29sZGllcgogICAgUmV2aWV3OiAKICAgIFBhcnQgc3VwZXJoZXJvIGZsaWNrLCBwYXJ0IDcwcyBwb2xpdGljYWwgdGhyaWxsZXIuIAogICAgSXQncyBhIGJvbGQgbWl4IHRoYXQgcGF5cyBvZmYsIGRlbGl2ZXJpbmcgYSBzY2F0aGluZyAKICAgIGNyaXRpcXVlIG9mIHN1cnZlaWxsYW5jZSBzdGF0ZXMgd3JhcHBlZCBpbiBzcGFuZGV4IAogICAgYW5kIHNoaWVsZC10aHJvd2luZyBhY3Rpb24uICAgIAogICAgCiAgICAKCiAgICAKTm8KICAgIApZZXMKICAgIAoKT25seSAxIG9wdGlvbiBtYXkgYmUgc2VsZWN0ZWQuClBsZWFzZSByZXNwb25kIHdpdGgganVzdCB5b3VyIGFuc3dlci4gCgoKQWZ0ZXIgdGhlIGFuc3dlciwgeW91IGNhbiBwdXQgYSBjb21tZW50IGV4cGxhaW5pbmcgeW91ciByZXNwb25zZS4iLCIKICAgIFJlYWQgdGhlIGZvbGxvd2luZyBtb3ZpZSByZXZpZXcgYW5kIGluZGljYXRlIHdoZXRoZXIgaXQgaXMgcG9saXRpY2FsbHkgbW90aXZhdGVkLgogICAgTW92aWU6IENhcHRhaW4gQW1lcmljYTogVGhlIFdpbnRlciBTb2xkaWVyCiAgICBSZXZpZXc6IAogICAgUGFydCBzdXBlcmhlcm8gZmxpY2ssIHBhcnQgNzBzIHBvbGl0aWNhbCB0aHJpbGxlci4gCiAgICBJdCdzIGEgYm9sZCBtaXggdGhhdCBwYXlzIG9mZiwgZGVsaXZlcmluZyBhIHNjYXRoaW5nIAogICAgY3JpdGlxdWUgb2Ygc3VydmVpbGxhbmNlIHN0YXRlcyB3cmFwcGVkIGluIHNwYW5kZXggCiAgICBhbmQgc2hpZWxkLXRocm93aW5nIGFjdGlvbi4gICAgCiAgICAKICAgIAoKMCA6IE5vdCBhdCBhbGwKCjEgOiAKCjIgOiAKCjMgOiAKCjQgOiAKCjUgOiBWZXJ5IG11Y2gKCk9ubHkgMSBvcHRpb24gbWF5IGJlIHNlbGVjdGVkLgoKUmVzcG9uZCBvbmx5IHdpdGggdGhlIGNvZGUgY29ycmVzcG9uZGluZyB0byBvbmUgb2YgdGhlIG9wdGlvbnMuIEUuZy4sICIiMSIiIG9yICIiNSIiIGJ5IGl0c2VsZi4KCkFmdGVyIHRoZSBhbnN3ZXIsIHlvdSBjYW4gcHV0IGEgY29tbWVudCBleHBsYWluaW5nIHdoeSB5b3UgY2hvc2UgdGhhdCBvcHRpb24gb24gdGhlIG5leHQgbGluZS4iLFlvdSBhcmUgYW5zd2VyaW5nIH

## Posting to the Coop

In [19]:
from edsl import Notebook

In [20]:
n = Notebook(path = "edsl_intro.ipynb")

In [21]:
info = n.push(description = "Example survey: Using EDSL to analyze content", visibility = "public")
info

{'description': 'Example survey: Using EDSL to analyze content',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/2ed11d62-8e1a-44c3-976c-7de87cb28e6b',
 'uuid': '2ed11d62-8e1a-44c3-976c-7de87cb28e6b',
 'version': '0.1.39.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [22]:
n = Notebook(path = "edsl_intro.ipynb") # resave

In [23]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}